In [ ]:
import os
import PyPDF2
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')
from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
input_directory = "Sources/"
num_pdfs = 20
labels = ["cat1", "cat2", "cat3"]

vectorizer = TfidfVectorizer(max_features=1000)

y = np.array([i % len(labels) for i in range(num_pdfs)])


Recibe un texto en español como entrada y realiza lematización de palabras. Utiliza el algoritmo Snowball para el español para lematizar cada palabra del texto y luego devuelve el texto lematizado como una sola cadena de palabras.

In [ ]:
def lemmatize_text(text):
    snowball = SnowballStemmer('spanish') 
    words = word_tokenize(text)
    lemmatized_words = [snowball.stem(word) for word in words]
    return " ".join(lemmatized_words)

La función `load_lexicon` carga el contenido de un archivo de léxico ubicado en la ruta especificada y lo devuelve como una cadena de texto.

In [ ]:
def load_lexicon(lexicon_file_path):
    with open(lexicon_file_path, 'r', encoding='utf-8') as file:
        lexicon_text = file.read()
    return lexicon_text

La función `is_related_to_lexicon` verifica si el texto contiene al menos una palabra que está presente en el léxico proporcionado como entrada. Retorna `True` si hay palabras en común entre el texto y el léxico, y `False` en caso contrario.

In [ ]:
def is_related_to_lexicon(text, lexicon):
    words_in_lexicon = set(lexicon.split())
    words_in_text = set(word_tokenize(text))
    return len(words_in_text.intersection(words_in_lexicon)) > 0


La función `load_lemmatized_text` carga el contenido de un archivo de texto lematizado ubicado en la ruta especificada y lo devuelve como una cadena de texto.

In [ ]:
def load_lemmatized_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lemmatized_text = file.read()
    return lemmatized_text

La función `get_most_common_words` recibe una lista de textos lematizados como entrada y devuelve las palabras más comunes presentes en esos textos. Puede especificarse el número de palabras más comunes a obtener (por defecto son 50). Utiliza expresiones regulares para extraer las palabras de los textos y la clase `Counter` para contar la frecuencia de cada palabra. Luego, selecciona las palabras más comunes y las devuelve en una lista.

In [ ]:
def get_most_common_words(lemmatized_texts, num_words=50):
    words_freq = Counter()
    for text in lemmatized_texts:
        words_freq.update(re.findall(r'\w+', text))

    most_common_words = [word for word, _ in words_freq.most_common(num_words)]
    return most_common_words


La función `calculate_topic_scores` calcula las puntuaciones de los temas en función de las palabras clave proporcionadas. Toma como entrada un texto lematizado y un diccionario que mapea cada tema a una lista de palabras clave asociadas a ese tema. Luego, cuenta la frecuencia de palabras en el texto lematizado y suma las frecuencias de las palabras clave de cada tema para obtener las puntuaciones respectivas. Finalmente, devuelve un diccionario que asigna cada tema a su puntuación correspondiente.

In [ ]:
def calculate_topic_scores(lemmatized_text, topic_keywords):
    word_freq = Counter(re.findall(r'\w+', lemmatized_text))
    topic_scores = {topic: sum(word_freq[word] for word in keywords) for topic, keywords in topic_keywords.items()}
    return topic_scores



La función `get_most_common_words` recibe una lista de textos lematizados como entrada y devuelve una lista de las palabras más comunes presentes en esos textos. Puede especificarse el número de palabras más comunes a obtener (por defecto son 50). Utiliza expresiones regulares para extraer las palabras de los textos y la clase `Counter` para contar la frecuencia de cada palabra. Luego, selecciona las palabras más comunes y las devuelve en una lista ordenada por frecuencia descendente.

In [ ]:
def get_most_common_words(lemmatized_texts, num_words=50):
    words_freq = Counter()
    for text in lemmatized_texts:
        words_freq.update(re.findall(r'\w+', text))

    most_common_words = [word for word, _ in words_freq.most_common(num_words)]
    return most_common_words


La función `calculate_topic_scores` calcula las puntuaciones de temas basadas en las palabras clave proporcionadas. Toma como entrada un texto lematizado y un diccionario que asocia cada tema con una lista de palabras clave relacionadas a ese tema. Luego, cuenta la frecuencia de las palabras en el texto lematizado y suma las frecuencias de las palabras clave correspondientes a cada tema. Finalmente, devuelve un diccionario que mapea cada tema a su puntuación resultante.

In [ ]:

def calculate_topic_scores(lemmatized_text, topic_keywords):
    word_freq = Counter(re.findall(r'\w+', lemmatized_text))
    topic_scores = {topic: sum(word_freq[word] for word in keywords) for topic, keywords in topic_keywords.items()}
    return topic_scores


La función `discretize_scores` toma un diccionario de puntuaciones de temas como entrada y devuelve un nuevo diccionario que asigna una etiqueta discreta (Positivo, Neutro o Negativo) a cada documento basado en la puntuación más alta obtenida en los temas. Si la puntuación más alta es mayor a 0, el documento se etiqueta como "Positivo"; si es igual a 0, se etiqueta como "Neutro"; y si es menor a 0, se etiqueta como "Negativo". El resultado final es un diccionario que mapea el número de cada documento a su etiqueta correspondiente.

In [ ]:
def discretize_scores(scores):
    discretized_scores = {}
    for doc_num, doc_scores in enumerate(scores, 1):
        max_score_topic = max(doc_scores, key=doc_scores.get)
        max_score = doc_scores[max_score_topic]
        if max_score > 0:
            discretized_scores[f"Documento {doc_num}"] = "Positivo"
        elif max_score == 0:
            discretized_scores[f"Documento {doc_num}"] = "Neutro"
        else:
            discretized_scores[f"Documento {doc_num}"] = "Negativo"
    return discretized_scores


La función `calculate_document_scores` calcula las puntuaciones de temas para una lista de textos lematizados. Toma como entrada una lista de textos lematizados y un diccionario de palabras clave asociadas a cada tema. Utiliza la función `calculate_topic_scores` para obtener las puntuaciones de temas para cada texto lematizado. Luego, retorna una lista que contiene los resultados de las puntuaciones de temas para cada texto lematizado en el mismo orden en el que fueron proporcionados.

In [ ]:
def calculate_document_scores(lemmatized_texts, topic_keywords):
    document_scores = []
    for lemmatized_text in lemmatized_texts:
        scores = calculate_topic_scores(lemmatized_text, topic_keywords)
        document_scores.append(scores)
    return document_scores

La función `calculate_average_scores` calcula el promedio de las puntuaciones de temas para cada documento. Toma como entrada una lista de diccionarios que representan las puntuaciones de temas para diferentes documentos. Para cada diccionario de puntuaciones, calcula el promedio de las puntuaciones y lo agrega a una lista. Finalmente, devuelve una lista que contiene los promedios de las puntuaciones de temas para cada documento en el mismo orden en el que fueron proporcionados.

In [ ]:
def calculate_average_scores(document_scores):
    average_scores = []
    for doc_scores in document_scores:
        average_score = sum(doc_scores.values()) / len(doc_scores)
        average_scores.append(average_score)
    return average_scores

La función `train_and_evaluate_models` entrena y evalúa dos modelos de aprendizaje (model1 y model2) utilizando datos de entrenamiento y prueba. Primero, entrena model1 con los datos de entrenamiento y luego hace predicciones en los datos de prueba para calcular su precisión (accuracy1). Luego, realiza una validación cruzada con cv pliegues en model2 utilizando los datos de entrenamiento para obtener la precisión promedio (mean_accuracy_model2). Finalmente, devuelve ambas precisiones como resultado.

In [ ]:
def train_and_evaluate_models(X_train, X_test, y_train, y_test, model1, model2, cv=5):

    model1.fit(X_train, y_train)
    y_pred1 = model1.predict(X_test)
    accuracy1 = accuracy_score(y_test, y_pred1)


    scores_model2 = cross_val_score(model2, X_train, y_train, cv=cv)
    mean_accuracy_model2 = scores_model2.mean()
    return accuracy1, mean_accuracy_model2


La función `extract_text_from_pdf` extrae el texto de un archivo PDF ubicado en la ruta especificada. Abre el archivo PDF, recorre cada página y concatena el texto extraído de todas las páginas en una sola cadena. Luego, devuelve el texto extraído como resultado.

In [ ]:
def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

La función `load_data` carga texto desde una serie de archivos PDF ubicados en un directorio de entrada especificado. Utiliza la función `extract_text_from_pdf` para extraer el texto de cada PDF y lo almacena en una lista llamada `data`. Luego, el código utiliza una variable `vectorizer` para transformar la lista `data` en una matriz de características `X` mediante algún método de vectorización (por ejemplo, conteo de palabras o TF-IDF). El resultado es la matriz de características `X`, que se puede utilizar para entrenar modelos de aprendizaje automático.

In [ ]:

def load_data(input_directory, num_pdfs):
    data = []
    for pdf_number in range(1, num_pdfs + 1):
        pdf_file_path = os.path.join(input_directory, f"{pdf_number}.pdf")
        text = extract_text_from_pdf(pdf_file_path)
        if text is not None:
            data.append(text)
    return data



In [ ]:

if __name__ == "__main__":
    input_directory = "Sources/"
    output_directory = "lematizacion"
    lexicon_file = "lexicon.txt"

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    lexicon = load_lexicon(lexicon_file)

    for pdf_number in range(1, 21): 
        pdf_file_path = os.path.join(input_directory, f"{pdf_number}.pdf")
        text = extract_text_from_pdf(pdf_file_path)

        if text is not None:  # Validar si el texto es diferente de None
            if is_related_to_lexicon(text, lexicon):
                lemmatized_text = lemmatize_text(text)

                lemmatized_output_file_path = os.path.join(output_directory, f"lematizacion_{pdf_number}.txt")
                with open(lemmatized_output_file_path, "w", encoding="utf-8") as file:
                    file.write(lemmatized_text)

    print("Se ha finalizado la lematización de los PDFs relacionados con el léxico y se han almacenado los resultados en la carpeta llamada 'lemmatization'.")


In [ ]:
lemmatization_directory = "lematizacion"
    
    # Obtener los archivos lematizados
lemmatized_texts = []
for file_name in os.listdir(lemmatization_directory):
    file_path = os.path.join(lemmatization_directory, file_name)
    lemmatized_text = load_lemmatized_text(file_path)
    lemmatized_texts.append(lemmatized_text)

    # Obtener las palabras clave para cada tema de las palabras más utilizadas
num_keywords_per_topic = 3
most_common_words = get_most_common_words(lemmatized_texts)
topic_keywords = {
    "Tema 1": most_common_words[:num_keywords_per_topic],
    "Tema 2": most_common_words[num_keywords_per_topic:num_keywords_per_topic*2],
    "Tema 3": most_common_words[num_keywords_per_topic*2:num_keywords_per_topic*3]
        
}

topic_scores_summary = {topic: 0 for topic in topic_keywords}

for lemmatized_text in lemmatized_texts:
    scores = calculate_topic_scores(lemmatized_text, topic_keywords)

    for topic, score in scores.items():
        topic_scores_summary[topic] += score

print("Puntuación acumulada para cada tema:")
for topic, score in topic_scores_summary.items():
     print(f"{topic}: {score}")

In [ ]:
lemmatization_directory = "lematizacion"
    
    # Obtener los archivos lematizados
lemmatized_texts = []
for file_name in os.listdir(lemmatization_directory):
    file_path = os.path.join(lemmatization_directory, file_name)
    lemmatized_text = load_lemmatized_text(file_path)
    lemmatized_texts.append(lemmatized_text)

    # Obtener las palabras clave para cada tema de las palabras más utilizadas
num_keywords_per_topic = 3
most_common_words = get_most_common_words(lemmatized_texts)
topic_keywords = {
    "Tema 1": most_common_words[:num_keywords_per_topic],
    "Tema 2": most_common_words[num_keywords_per_topic:num_keywords_per_topic*2],
    "Tema 3": most_common_words[num_keywords_per_topic*2:num_keywords_per_topic*3]
        
 }

topic_scores_summary = {topic: 0 for topic in topic_keywords}
num_files = len(lemmatized_texts)

for lemmatized_text in lemmatized_texts:
    scores = calculate_topic_scores(lemmatized_text, topic_keywords)

    for topic, score in scores.items():
        topic_scores_summary[topic] += score / num_files

print("Puntaje promedio por Tema:")
for topic, score in topic_scores_summary.items():
        print(f"{topic}: {score}")

In [ ]:
document_scores = calculate_document_scores(lemmatized_texts, topic_keywords)

print("Puntaje acumulado de cada documento:")
for doc_num, scores in enumerate(document_scores, 1):
    print(f"Documento {doc_num}: {scores}")

In [ ]:
document_scores = calculate_document_scores(lemmatized_texts, topic_keywords)


average_scores = calculate_average_scores(document_scores)

print("Puntaje promedio para cada documento:")
for doc_num, avg_score in enumerate(average_scores, 1):
    print(f"Documento {doc_num}: {avg_score}")

In [ ]:
document_scores = calculate_document_scores(lemmatized_texts, topic_keywords)


discretized_scores = discretize_scores(document_scores)

print("Discretización de los puntajes:")
for doc, category in discretized_scores.items():
    print(f"{doc}: {category}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model1 = MultinomialNB()
model2 = RandomForestClassifier()

accuracy_test_train, mean_accuracy_cv = train_and_evaluate_models(X_train, X_test, y_train, y_test, model1, model2, cv=5)

print("Precisión en testing & training:", accuracy_test_train)
print("Precisión en cross-validation:", mean_accuracy_cv)